In [1]:
# =============================================================================
# Step 1: Load and Inspect Data
# =============================================================================

# Import the pandas library for data manipulation
import pandas as pd

# --- Load the Dataset ---
# We define the file path relative to our notebook's location.
# Our notebook is in '02_notebooks/', and the data is in '01_data/raw/'.
# So we need to go one level up ('../') and then into the data folder.
file_path = '../01_data/raw/online_retail_II.xlsx'

# Load the Excel file into a pandas DataFrame
# We'll load the 'Year 2009-2010' sheet as discussed.
try:
    retail_df = pd.read_excel(file_path, sheet_name='Year 2009-2010')
    print("✅ Dataset loaded successfully!")
except FileNotFoundError:
    print(f"❌ Error: The file '{file_path}' was not found. Please ensure the path is correct.")

# --- Initial Inspection ---
# Let's perform a basic check to understand the data we've loaded.

# 1. Display the first 5 rows of the dataframe
print("\nFirst 5 rows of the dataset:")
display(retail_df.head())

# 2. Display the shape of the dataframe (rows, columns)
print(f"\nDataset shape (rows, columns): {retail_df.shape}")

# 3. Display a concise summary of the dataframe, including column data types and non-null counts
print("\nDataset Info:")
retail_df.info()

✅ Dataset loaded successfully!

First 5 rows of the dataset:


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom



Dataset shape (rows, columns): (525461, 8)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [2]:
# =============================================================================
# Step 2: Data Cleaning
# =============================================================================
import pandas as pd

# We are assuming 'retail_df' is already loaded from the previous step.

# --- 2.1 Handle Missing Values ---

# First, let's see how many missing values are in each column.
print("--- 2.1 Handling Missing Values ---")
missing_values = retail_df.isnull().sum()
print("Missing values per column:\n", missing_values)
print("-" * 30)

# The 'Customer ID' is crucial for our goal of personalized marketing.
# If the ID is missing, we cannot attribute the purchase to any specific customer.
# Therefore, the best strategy is to remove rows where 'Customer ID' is null.
rows_before = retail_df.shape[0]
retail_df.dropna(subset=['Customer ID'], inplace=True)
rows_after = retail_df.shape[0]
print(f"Removed {rows_before - rows_after} rows with missing 'Customer ID'.")
print(f"Dataset shape after dropping rows: {retail_df.shape}")
print("-" * 30)


# --- 2.2 Correct Data Types ---

# 'Customer ID' should be treated as a whole number (integer) but since pandas
# might have it as a float due to the previous nulls, let's convert it.
print("\n--- 2.2 Correcting Data Types ---")
retail_df['Customer ID'] = retail_df['Customer ID'].astype(int)
print("Converted 'Customer ID' to integer type.")

# 'InvoiceDate' is critical for any time-based analysis (like finding the last purchase date).
# Let's convert it from text to a proper datetime object.
retail_df['InvoiceDate'] = pd.to_datetime(retail_df['InvoiceDate'])
print("Converted 'InvoiceDate' to datetime type.")
print("-" * 30)


# --- 2.3 Handle Canceled Orders ---

# We might see that some 'Invoice' numbers start with 'C', indicating a cancellation.
# These transactions often have a negative 'Quantity'. Let's check for negative quantities.
print("\n--- 2.3 Handling Canceled Orders & Negative Quantities ---")
print(f"Number of rows with negative quantity: {(retail_df['Quantity'] <= 0).sum()}")

# For our analysis of purchase behavior, we only want to look at actual purchases, not returns.
# So, we will remove all rows where the quantity is not positive.
retail_df = retail_df[retail_df['Quantity'] > 0]
print("Removed rows with non-positive quantity.")
print(f"Dataset shape after removing returns: {retail_df.shape}")
print("-" * 30)


# --- 2.4 Handle Zero Prices ---

# Sometimes, items are listed with a price of 0.0. These could be errors or promotional items
# that don't represent a real purchase. Let's check for them.
print("\n--- 2.4 Handling Zero Prices ---")
print(f"Number of rows with zero price: {(retail_df['Price'] == 0).sum()}")

# We will remove these as they don't contribute to sales analysis.
retail_df = retail_df[retail_df['Price'] > 0]
print("Removed rows with zero price.")
print(f"Dataset shape after removing zero price items: {retail_df.shape}")
print("-" * 30)


# --- Final Check ---
# Let's look at the info and a statistical summary of our now-cleaned dataframe.
print("\n--- Final Check on Cleaned Data ---")
print("\nFinal Dataset Info:")
retail_df.info()

print("\nFinal Dataset Statistical Summary:")
display(retail_df.describe())

--- 2.1 Handling Missing Values ---
Missing values per column:
 Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64
------------------------------
Removed 107927 rows with missing 'Customer ID'.
Dataset shape after dropping rows: (417534, 8)
------------------------------

--- 2.2 Correcting Data Types ---
Converted 'Customer ID' to integer type.
Converted 'InvoiceDate' to datetime type.
------------------------------

--- 2.3 Handling Canceled Orders & Negative Quantities ---
Number of rows with negative quantity: 9839
Removed rows with non-positive quantity.
Dataset shape after removing returns: (407695, 8)
------------------------------

--- 2.4 Handling Zero Prices ---
Number of rows with zero price: 31
Removed rows with zero price.
Dataset shape after removing zero price items: (407664, 8)
------------------------------

--- Final Check on Cleane

,Quantity,InvoiceDate,Price,Customer ID
count,407664.000000,407664,407664.000000,407664.000000
mean,13.585585,2010-07-01 10:15:11.871688192,3.294438,15368.592598
min,1.000000,2009-12-01 07:45:00,0.001000,12346.000000
25%,2.000000,2010-03-26 14:01:00,1.250000,13997.000000
50%,5.000000,2010-07-09 15:47:00,1.950000,15321.000000
75%,12.000000,2010-10-14 17:09:00,3.750000,16812.000000
max,19152.000000,2010-12-09 20:01:00,10953.500000,18287.000000
std,96.840747,NaN,34.757965,1679.762138


In [3]:
# =============================================================================
# Step 3: Feature Engineering (RFM Analysis)
# =============================================================================
import datetime as dt

# --- 3.1 Create a 'TotalPrice' Column ---
# This is a required feature for calculating the Monetary value.
print("--- 3.1 Engineering 'TotalPrice' Feature ---")
retail_df['TotalPrice'] = retail_df['Quantity'] * retail_df['Price']
print("Created 'TotalPrice' column.")
display(retail_df.head())
print("-" * 30)


# --- 3.2 Calculate RFM Values for each Customer ---
print("\n--- 3.2 Calculating RFM Values ---")

# To calculate Recency, we need a "snapshot" date. This will be the day after
# the last transaction in the dataset.
snapshot_date = retail_df['InvoiceDate'].max() + dt.timedelta(days=1)
print(f"Snapshot date for Recency calculation: {snapshot_date}")

# Group data by each customer
# We will calculate Recency, Frequency, and Monetary value for each Customer ID
rfm_df = retail_df.groupby('Customer ID').agg({
    'InvoiceDate': lambda date: (snapshot_date - date.max()).days, # Recency
    'Invoice': 'nunique',                                           # Frequency (count of unique invoices)
    'TotalPrice': 'sum'                                             # Monetary (sum of all purchases)
})

# Rename the columns to be more descriptive
rfm_df.rename(columns={'InvoiceDate': 'Recency',
                       'Invoice': 'Frequency',
                       'TotalPrice': 'MonetaryValue'}, inplace=True)

print("\nCalculated RFM values for each customer.")
print("Displaying the first 5 rows of the RFM DataFrame:")
display(rfm_df.head())
print("-" * 30)

# --- Final Check on RFM Data ---
print("\n--- Final Check on RFM DataFrame ---")
print("\nRFM DataFrame Info:")
rfm_df.info()

print("\nRFM DataFrame Statistical Summary:")
display(rfm_df.describe())

--- 3.1 Engineering 'TotalPrice' Feature ---
Created 'TotalPrice' column.


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


------------------------------

--- 3.2 Calculating RFM Values ---
Snapshot date for Recency calculation: 2010-12-10 20:01:00

Calculated RFM values for each customer.
Displaying the first 5 rows of the RFM DataFrame:


,Recency,Frequency,MonetaryValue
Customer ID,,,
12346,165,11,372.86
12347,3,2,1323.32
12348,74,1,222.16
12349,43,3,2671.14
12351,11,1,300.93


------------------------------

--- Final Check on RFM DataFrame ---

RFM DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 4312 entries, 12346 to 18287
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Recency        4312 non-null   int64  
 1   Frequency      4312 non-null   int64  
 2   MonetaryValue  4312 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 117.9 KB

RFM DataFrame Statistical Summary:


,Recency,Frequency,MonetaryValue
count,4312.000000,4312.000000,4312.000000
mean,91.171846,4.455705,2048.238236
std,96.860633,8.170213,8914.481280
min,1.000000,1.000000,2.950000
25%,18.000000,1.000000,307.987500
50%,53.000000,2.000000,706.020000
75%,136.000000,5.000000,1723.142500
max,374.000000,205.000000,349164.350000


In [5]:
# =============================================================================
# Step 4: Creating RFM Segments (Corrected Version)
# =============================================================================

# --- 4.1 Calculate RFM Scores based on Quartiles ---
print("--- 4.1 Calculating RFM Scores ---")

# Create labels for the scores
r_labels = range(4, 0, -1) # For Recency, lower is better (4 is best)
f_labels = range(1, 5)    # For Frequency, higher is better (4 is best)
m_labels = range(1, 5)    # For Monetary, higher is better (4 is best)

# Use pd.qcut to cut the data into 4 equal parts (quartiles) and assign scores
# We use .rank(method='first') to handle the duplicate values in Frequency and MonetaryValue
# which caused the error. This ensures each value has a unique rank.

rfm_df['R_score'] = pd.qcut(rfm_df['Recency'], q=4, labels=r_labels)
rfm_df['F_score'] = pd.qcut(rfm_df['Frequency'].rank(method='first'), q=4, labels=f_labels)
rfm_df['M_score'] = pd.qcut(rfm_df['MonetaryValue'].rank(method='first'), q=4, labels=m_labels)

print("Assigned R, F, M scores to each customer.")
display(rfm_df.head())
print("-" * 30)


# --- 4.2 Combine Scores to Create Segments ---
print("\n--- 4.2 Combining scores to create segments ---")

rfm_df['RFM_Segment'] = rfm_df.apply(lambda row: str(row['R_score']) + str(row['F_score']) + str(row['M_score']), axis=1)
rfm_df['RFM_Score'] = rfm_df['R_score'].astype(int) + rfm_df['F_score'].astype(int) + rfm_df['M_score'].astype(int)

print("Created RFM_Segment and RFM_Score columns.")
display(rfm_df.head())
print("-" * 30)

# --- 4.3 (Optional but Recommended) Save the Cleaned Data ---
print("\n--- 4.3 Saving processed data ---")
cleaned_data_path = '../01_data/processed/cleaned_retail_data.csv'
rfm_data_path = '../01_data/processed/rfm_customer_data.csv'

# We need to save the retail_df from the previous step which has TotalPrice
retail_df.to_csv(cleaned_data_path, index=False)
rfm_df.to_csv(rfm_data_path)

print(f"✅ Cleaned transaction data saved to: {cleaned_data_path}")
print(f"✅ RFM customer data saved to: {rfm_data_path}")

--- 4.1 Calculating RFM Scores ---
Assigned R, F, M scores to each customer.


,Recency,Frequency,MonetaryValue,R_score,F_score,M_score
Customer ID,,,,,,
12346,165,11,372.86,1,4,2
12347,3,2,1323.32,4,2,3
12348,74,1,222.16,2,1,1
12349,43,3,2671.14,3,3,4
12351,11,1,300.93,4,1,1


------------------------------

--- 4.2 Combining scores to create segments ---
Created RFM_Segment and RFM_Score columns.


,Recency,Frequency,MonetaryValue,R_score,F_score,M_score,RFM_Segment,RFM_Score
Customer ID,,,,,,,,
12346,165,11,372.86,1,4,2,1.04.02.0,7
12347,3,2,1323.32,4,2,3,4.02.03.0,9
12348,74,1,222.16,2,1,1,2.01.01.0,4
12349,43,3,2671.14,3,3,4,3.03.04.0,10
12351,11,1,300.93,4,1,1,4.01.01.0,6


------------------------------

--- 4.3 Saving processed data ---
✅ Cleaned transaction data saved to: ../01_data/processed/cleaned_retail_data.csv
✅ RFM customer data saved to: ../01_data/processed/rfm_customer_data.csv


In [8]:
# =============================================================================
# Step 5 (Revised): Analyze and Name RFM Segments using RFM Score
# =============================================================================

# --- 5.1 Define Segment Names based on the total RFM Score ---
print("--- 5.1 Defining segment names using total RFM Score---")

# Define the score boundaries and the corresponding names.
# For example, a score of 10-12 is our best group, 3-5 is the worst.
score_bins = [0, 6, 8, 10, 12] # Bins: (0-6], (6-8], (8-10], (10-12]
score_labels = ['Hibernating', 'Needs Attention', 'Loyal Customers', 'Champions']

# Use pd.cut to create the 'Segment' column based on RFM_Score
rfm_df['Segment'] = pd.cut(rfm_df['RFM_Score'], bins=score_bins, labels=score_labels)

print("Assigned descriptive names to each segment based on score.")
display(rfm_df.head())
print("-" * 30)


# --- 5.2 Analyze the Segments ---
print("\n--- 5.2 Analyzing segment characteristics ---")

# Group by the new 'Segment' column and calculate key metrics
segment_analysis = rfm_df.groupby('Segment', observed=True).agg(
    Recency_mean=('Recency', 'mean'),
    Frequency_mean=('Frequency', 'mean'),
    MonetaryValue_mean=('MonetaryValue', 'mean'),
    Customer_Count=('Recency', 'count')
).round(1)

print("Summary of customer segments:")
display(segment_analysis.sort_values(by='MonetaryValue_mean', ascending=False))

# --- 5.3 (Final Save) Save the fully segmented data ---
print("\n--- 5.3 Saving final segmented data ---")
final_data_path = '../01_data/processed/final_customer_segments.csv'
rfm_df.to_csv(final_data_path)

print(f"✅ Final segmented customer data saved to: {final_data_path}")

--- 5.1 Defining segment names using total RFM Score---
Assigned descriptive names to each segment based on score.


,Recency,Frequency,MonetaryValue,R_score,F_score,M_score,RFM_Segment,RFM_Score,Segment
Customer ID,,,,,,,,,
12346,165,11,372.86,1,4,2,1.04.02.0,7,Needs Attention
12347,3,2,1323.32,4,2,3,4.02.03.0,9,Loyal Customers
12348,74,1,222.16,2,1,1,2.01.01.0,4,Hibernating
12349,43,3,2671.14,3,3,4,3.03.04.0,10,Loyal Customers
12351,11,1,300.93,4,1,1,4.01.01.0,6,Hibernating


------------------------------

--- 5.2 Analyzing segment characteristics ---
Summary of customer segments:


,Recency_mean,Frequency_mean,MonetaryValue_mean,Customer_Count
Segment,,,,
Champions,14.1,12.6,6818.3,841
Loyal Customers,42.9,4.8,1972.0,830
Needs Attention,71.3,2.6,958.2,909
Hibernating,162.2,1.3,340.6,1732



--- 5.3 Saving final segmented data ---
✅ Final segmented customer data saved to: ../01_data/processed/final_customer_segments.csv
